In [74]:
import pandas as pd
import numpy as np
import gspread # https://github.com/burnash/gspread

In [75]:
from pathlib import Path

## Read SpreadSheet

In [76]:
gc = gspread.service_account(filename=str(PARENT.joinpath("client_secret.json")))
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1uAi-Yfq-rJl_cqQaVe9Fv1rLlBJcmEtDpUB0NTOrLAs/edit#gid=625331269")

In [77]:
# Get a list of all worksheets
worksheet_list = sh.worksheets()
print(worksheet_list)

[<Worksheet 'set_de_datos_unificado' id:625331269>, <Worksheet 'Listados de Validación' id:889279027>]


In [78]:
# By title
worksheet = sh.worksheet("set_de_datos_unificado")

In [79]:
raw_data = worksheet.get_all_values()

In [80]:
df = pd.DataFrame(raw_data[1:], columns=raw_data[0])
df

,N,NRO_REGISTRO,FECHA_RESOLUCION,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,...,DECISION_DE_ADMISIBILIDAD_TSJ,N_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN
0,1,,1_8_2016,Pablo_Casas,penal,1,ley_14346,proteccion_animal,simples,no,...,,,,,,,,,,
1,2,,1_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,,,,,,,,,,
2,3,,2_8_2016,Pablo_Casas,contravencional,52,codigo_contravencional,hostigamiento,agravado_violencia_de_genero,si,...,,,,,,,,,,
3,4,,2_8_2016,Pablo_Casas,contravencional,73,codigo_contravencional,incumplir_clausura,,no,...,,,,,,,,,,
4,5,,2_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836,,,,,,,,,,,...,,,,,,,,,,
3837,,,,,,,,,,,...,,,,,,,,,,
3838,,,,,,,,,,,...,,,,,,,,,,
3839,,,,,,,,,,,...,,,,,,,,,,


In [81]:
# VERIFY LATER
df.replace('', np.nan, inplace=True)
df.dropna(axis=0, how="all", subset=["N", "NRO_REGISTRO"], inplace=True)

In [82]:
df.shape

(3405, 61)

In [83]:
df['LINK'][1000]

'https://drive.google.com/open?id=1bbf2UrvoaeCgaGURpaXCI0Y8wuPnxDXR'

In [43]:
document_ids = [link.split('id=') for link in df['LINK'] if type(link) == str]
document_ids

[['https://drive.google.com/open?', '0B9wNhp3GjjazTVhZM2dCRU1IN2M'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazTjRHVHl2YzBtUEU'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazY3VCak5kdTBvWWc'],
 ['https://drive.google.com/open?', '0B9wNhp3Gjjazc2V4eWVaeWVRQUE'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazZFdmVXlWbFdYSHc'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazZ2xEYWQ3UHdlNWs'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazamJQMjJaYUU1Ym8'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazdHVtLTVpS2JWNms'],
 ['https://drive.google.com/open?', '0B9wNhp3Gjjazc2loTFlxTFdhZmM'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazUG5XOWNmOVNob1U'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazV19Fa0Y3am9QUkk'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazbW9OM2UwdEVTczQ'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazbGJXckc3S05sVTQ'],
 ['https://drive.google.com/open?', '0B9wNhp3GjjazT3dHRWZDdTQzODA'],
 ['https://drive.google.com/open?'

## Get PDF content 

https://developers.google.com/drive/api/v2/reference/files/get

https://developers.google.com/drive/api/v3/quickstart/python

In [137]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

# The ID of a sample document.
FILE_ID = "1UQP8YLmRUuHE8BhbUd6s9PTKTiNkJKuE"

In [138]:
from google.oauth2 import service_account
from apiclient import errors
from apiclient import http

#SCOPES = ['https://www.googleapis.com/auth/documents.readonly']
SERVICE_ACCOUNT_FILE = '../client_secret.json'

creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [139]:
#service = build('file', 'v1', credentials=credentials)
service = build('drive', 'v3', credentials=creds)

In [141]:
# metadata
info = service.files().get(fileId=FILE_ID).execute()

In [142]:
info

{'kind': 'drive#file',
 'id': '1UQP8YLmRUuHE8BhbUd6s9PTKTiNkJKuE',
 'name': '14238_06_02_17_128_pornografia_infantil',
 'mimeType': 'application/vnd.oasis.opendocument.text'}

In [146]:
info['name'][-3:]

'til'

In [119]:
request = service.files().get_media(fileId=FILE_ID)
#media_request = http.MediaIoBaseDownload('../', request)

In [129]:
fh = io.BytesIO()

In [ ]:
fh.

In [130]:
import io
fh = io.BytesIO()
#fh = "../" + service.files().get(fileId=FILE_ID).execute()['name']
downloader = http.MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [131]:
def write_bytesio_to_file(filename, bytesio):
    """
    Write the contents of the given BytesIO to a file.
    Creates the file or overwrites the file if it does
    not exist yet. 
    """
    with open(filename, "wb") as outfile:
        # Copy the BytesIO stream to the output file
        outfile.write(bytesio.getbuffer())

In [134]:
"../" + service.files().get(fileId=FILE_ID).execute()['name']

'../9498 P. D. s. ley de protección animal.pdf'

In [136]:
write_bytesio_to_file(str("../" + service.files().get(fileId=FILE_ID).execute()['name']), fh)

In [15]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [46]:
FILE_ID = document_ids[10][1]
if __name__ == "__main__":
    file_id = FILE_ID
    destination = '../file1.pdf'
    download_file_from_google_drive(file_id, destination)

In [85]:
os.remove('../file1.txt')

## Get text from PDF

https://towardsdatascience.com/how-to-extract-text-from-pdf-245482a96de7

In [49]:
import PyPDF2
fhandle = open(r'../file1.pdf', 'rb')
pdf = PyPDF2.PdfFileReader(fhandle)
# pagehandle = pdfReader.getPage(1)
# print(pagehandle.extractText())

In [53]:
with open('../file1.txt', 'w') as f:
    for page_num in range(pdf.numPages):
        pageObj = pdf.getPage(page_num)
        
        try:
            txt = pageObj.extractText()
        except:
            pass
        else:
            f.write(txt)
    f.close()

In [56]:
import pdfplumber
with pdfplumber.open(r'../file1.pdf') as pdf:
    first_page = pdf.pages[2]
    print(first_page.extract_text())

�  PodJeurd idceil aaCl i uddaeBd u enAoisr es
�  FuePreon Caoln traveydn ecF iaolntala s
MarAínat olm\lf:ilalonak  isi 
circunsptoalrna cc uiaasl ee  nomjuóc hdoa dqou en oh absíiad aSoeu ctroert8irziaad o 
pareal lpoe,rq ou ec omeol i mputlaeid nos irsetfiiór iqéuneedl ol loec atario 
Q___  \ _e ruan ap ersobnuae nQau.ed  ichpae rsopnear maneence illó u gpaorre  l 
_  \ �t érmdienu ona  fitoi,e mdpuor aenltc eu arle ciebldi ión edreoal l quiQlueer . 
=y '"! 
luegeoli, m putlaeed not rdeoglsól  avdeeas c ceasllo o cuanlda,e  l ap ueryta 
�il 
� lar estdaenclta en daqduoel  aa segurpaoblrao c, u asled  iriaglli uóg eanre  l 
t¡ saf 2i0o1 c3o rroboqruaeens dtoav baac ío. 
.i:1esi"'. -..
�,  Continusaurn edloa mteon,c ioqnuóep  orm otivpoesr sonales, 
U1TI  recipéundc oo ncuarlcr oimre recldi ío2a 8 d ej undieo2  01c6u,a naddov irtió 
e 
queel c andandoeo s taybq auc eo snu l lanvoep  odaíbar liapr u erAt raa.íd ze  
elltoo,c lóam  ismya e greusnóm  asculqiunioel nee  ntreelgc óo ntrdaet

In [91]:
var = "foo"
str(PARENT.joinpath("data/interim/" + var))

'/home/yabra/Desktop/gender-violence/data/interim/foo'

In [59]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('../file1.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()
print(text)

�  Poder Judicial 
�  Fuero Penal Contravcndonal 

de la Ciudad de Buenos Aires

y de Faltas 

///nos Aires, 4 de agosto de 2016. 

AUTOS Y VISTOS: 

Maria AnloMi8 Mm,rJqler.l

seci·��tpr\f!.

Para resolver 

en esta causa nº 9878/16; 

caratulada 

"L                  , 

R                

E                     

s/art. 11179: 181 1 Usurpación 

(Despojo) 

-CP (p/L 

2303)", 

en trámite 

ante este Juzgado en lo Penal, Contravencional 

y de Faltas 

Nº 1 O, en relación 

a la solicitud 

de incompetencia 

planteada 

por el Ministerio 

Público 

Fiscal. 

RESULTA: 

Que se inician 

las presentes 

actuaciones 

a raíz de la denuncia 

formulada 

por O        

 W        T          Á         
 el día 7 de julio del corriente 

año 

contra R        

E            

L           . 

En dicha ocasión refirió 

que era propietario 

del inmueble 

sito en 

la calle La Rioja ** de esta ciudad y que hacía tiempo que no la frecuentaba 

y 

que tenía planes de refaccionarlo 

con su 

In [60]:
text

'�  Poder Judicial \n�  Fuero Penal Contravcndonal \n\nde la Ciudad de Buenos Aires\n\ny de Faltas \n\n///nos Aires, 4 de agosto de 2016. \n\nAUTOS Y VISTOS: \n\nMaria AnloMi8 Mm,rJqler.l\n\nseci·��tpr\\f!.\n\nPara resolver \n\nen esta causa nº 9878/16; \n\ncaratulada \n\n"L                  , \n\nR                \n\nE                     \n\ns/art. 11179: 181 1 Usurpación \n\n(Despojo) \n\n-CP (p/L \n\n2303)", \n\nen trámite \n\nante este Juzgado en lo Penal, Contravencional \n\ny de Faltas \n\nNº 1 O, en relación \n\na la solicitud \n\nde incompetencia \n\nplanteada \n\npor el Ministerio \n\nPúblico \n\nFiscal. \n\nRESULTA: \n\nQue se inician \n\nlas presentes \n\nactuaciones \n\na raíz de la denuncia \n\nformulada \n\npor O        \n\n W        T          Á         \n el día 7 de julio del corriente \n\naño \n\ncontra R        \n\nE            \n\nL           . \n\nEn dicha ocasión refirió \n\nque era propietario \n\ndel inmueble \n\nsito en \n\nla calle La Rioja ** de esta ciudad 

In [93]:
unique_ids = set(df["LINK"])

In [96]:
unique_ids = df['LINK']

In [98]:
unique_ids.dropna(inplace=True)

0       https://drive.google.com/open?id=0B9wNhp3Gjjaz...
1       https://drive.google.com/open?id=0B9wNhp3Gjjaz...
2       https://drive.google.com/open?id=0B9wNhp3Gjjaz...
3       https://drive.google.com/open?id=0B9wNhp3Gjjaz...
4       https://drive.google.com/open?id=0B9wNhp3Gjjaz...
                              ...                        
3406    https://drive.google.com/file/d/1wVeBMKcdaSc4w...
3407    https://drive.google.com/file/d/1Zht2hB79c-c5d...
3408    https://drive.google.com/file/d/1Zht2hB79c-c5d...
3409    https://drive.google.com/file/d/1Zht2hB79c-c5d...
3410    https://drive.google.com/file/d/1w7qPcRIyqevAh...
Name: LINK, Length: 3398, dtype: object

In [95]:
for i in unique_ids:
    print(i)

nan
https://drive.google.com/open?id=1IgUOh033l9Vpgp0eGrZF6s2d3d1wXlOY
https://drive.google.com/open?id=0B9wNhp3GjjazYUFrTkxoeHpaM28
https://drive.google.com/open?id=1Ivd0RfqLvCTMN4iZWEYQ3l6WfxazNYjNJL_bUaTMWNs
https://drive.google.com/open?id=1xZkvQ3VS4GX1nzUtHEYBoyCVX_SuFD2J
https://drive.google.com/open?id=1ZWphZ4Y5W9d7hrUmEwayIML789QjO1nIb069fswdLHw
https://drive.google.com/file/d/1FdDWN0XHXrf9aKkNHJVvl8FlFkQPd58D/view?usp=sharing
https://drive.google.com/open?id=0B9wNhp3GjjazTTV2Y3JwazRXQWc
https://drive.google.com/open?id=1P2trppw3RGDx2_C-wpXBPYunozbpHsyV
https://drive.google.com/open?id=1G5oTGinz1TuAyH6LeTa5PGJYZoLqith4
https://drive.google.com/open?id=1Sd8EE2feF3oO3YNtRCbX5kIQ7cYsv1jxKrZAgQoPJ70
https://drive.google.com/open?id=1VBwFMBy4cuPCnO2FoIh3wY55JfjJJLfLS2am1vMW9d8
https://drive.google.com/open?id=14ejXalHzKADm3cnt-VaKCx-8aqi3p1Rm
https://docs.google.com/document/d/1sA7TgSok5Fl-alYqraQJ-AeMsiHz_d826HYC4N8S2LI/edit?usp=sharing
https://drive.google.com/open?id=1px6jPmtiH9Q

https://drive.google.com/open?id=19pOVXQMJ8IgJLKo0b6ctJCjdyP4LXP9W
https://drive.google.com/open?id=18v_NdPgga0MErFYlgGtnvwje_Y7muD2J
https://drive.google.com/open?id=1kaOdu9qPJt-nA6Ha9Y4lCxriqMmBQASE
https://drive.google.com/open?id=16_q8qktLjKC2EPSlZFQDx0gfLxVvuMwL
https://drive.google.com/open?id=15tJuvsZ9mRzz7ElRBQP7MeMYIAeOT_EA
https://drive.google.com/open?id=1AGy34r1Gef2iMeXfDwqqoQ6NlZaPZtdW
https://drive.google.com/open?id=1PRV-3om_vlUekqFPRray5yFfRZoi6uyU
https://drive.google.com/open?id=13Ks7bXPHTGTH6wXl0gl3lBVeiCBcNRaT
https://drive.google.com/open?id=1yj2D1P_DNWZu3SUHNh3JwU76nLMy6t4Q
https://drive.google.com/open?id=1zCf57YtXks1k-eA2MqYTiv320jQzpqM8
https://drive.google.com/open?id=1yxaawIJeeQfq7Buy70vFG7UzZao709l3
https://drive.google.com/open?id=0B9wNhp3GjjazT0tSYTA1Yy1zZmM
https://drive.google.com/open?id=11o8_8AzD-xij_pRPPHN9oPVnbO3bv_BD
https://drive.google.com/open?id=14OZX5UcLHmoAIrfRqc4omN77pkBd2QiI
https://drive.google.com/open?id=1VlxXs7nB3Z0bBf5mE0OgtuyaRlHDorbL


In [102]:
ids_set = set(unique_ids)

In [103]:
len(ids_set)

2738

In [108]:
df

,N,NRO_REGISTRO,FECHA_RESOLUCION,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,...,DECISION_DE_ADMISIBILIDAD_TSJ,N_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN
0,1,NaN,1_8_2016,Pablo_Casas,penal,1,ley_14346,proteccion_animal,simples,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,2_8_2016,Pablo_Casas,contravencional,52,codigo_contravencional,hostigamiento,agravado_violencia_de_genero,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,2_8_2016,Pablo_Casas,contravencional,73,codigo_contravencional,incumplir_clausura,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,2_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406,2783,2890,7_08_20,Pablo_Casas,penal,189bis,codigo_penal_de_la_nacion,tenencia_de_arma,de_fuego_uso_civil,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3407,2784,2891,8_08_20,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3408,NaN,2891,8_08_20,Pablo_Casas,penal,183,codigo_penal_de_la_nacion,daños,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3409,NaN,2891,8_08_20,Pablo_Casas,penal,186_inc1,codigo_penal_de_la_nacion,incendio_explosion_inundacion_con_peligro_para...,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
# test = "https://docs.google.com/document/d/1JOvMoPo0Gv4dWafvbeCRCUn9HP3C2l-Fv9r4dwLYm6E/edit?usp=sharing"
test = 

In [150]:
test.split('/document/d/')[1].split('/')

['1JOvMoPo0Gv4dWafvbeCRCUn9HP3C2l-Fv9r4dwLYm6E', 'edit?usp=sharing']